In [ ]:
import numpy as np
import random
import math

# Définir le nombre de lignes et de colonnes
LIGNES = 6
COLONNES = 12

joueur_humain = 1
joueur_ia = 2

# Ajout de nouvelles fonctions pour jouer
def placer_pion(plateau, ligne, colonne, pion):
    """Place un pion dans le tableau."""
    plateau[ligne][colonne] = pion

def prochaine_ligne_libre(plateau, colonne):
    """Trouve la prochaine ligne libre dans une colonne."""
    for r in range(LIGNES):
        if plateau[r][colonne] == 0:
            return r
    return None

def afficher_plateau(plateau):
    """Affiche le plateau de jeu."""
    for ligne in range(LIGNES-1, -1, -1):
        for colonne in range(COLONNES):
            print(int(plateau[ligne][colonne]), end=' ')
        print()
    print()

def coup_gagnant(plateau, pion):
    """Vérifie si un joueur a gagné."""
    # Vérifier les positions horizontales pour la victoire
    for c in range(COLONNES-3):
        for r in range(LIGNES):
            if plateau[r][c] == pion and plateau[r][c+1] == pion and plateau[r][c+2] == pion and plateau[r][c+3] == pion:
                return True

    # Vérifier les positions verticales pour la victoire
    for c in range(COLONNES):
        for r in range(LIGNES-3):
            if plateau[r][c] == pion and plateau[r+1][c] == pion and plateau[r+2][c] == pion and plateau[r+3][c] == pion:
                return True

    # Vérifier les diagonales positives pour la victoire
    for c in range(COLONNES-3):
        for r in range(LIGNES-3):
            if plateau[r][c] == pion and plateau[r+1][c+1] == pion and plateau[r+2][c+2] == pion and plateau[r+3][c+3] == pion:
                return True

    # Vérifier les diagonales négatives pour la victoire
    for c in range(COLONNES-3):
        for r in range(3, LIGNES):
            if plateau[r][c] == pion and plateau[r-1][c+1] == pion and plateau[r-2][c+2] == pion and plateau[r-3][c+3] == pion:
                return True

    return False

def colonnes_valides(plateau):
    """Retourne une liste de colonnes valides (non pleines)."""
    colonnes_valides = []
    for col in range(COLONNES):
        if plateau[LIGNES-1][col] == 0:
            colonnes_valides.append(col)
    return colonnes_valides

def evaluer_fenetre(fenetre, pion):
    """
        Permet d'etablir un score sur chaque fenetre de 4 cases consecutives, en ligne, en colonne et en diagonale.
        Utilise les pions presents sur la fenetre, ainsi que les cases vides pour determiner si un coup vaut le coup d'etre
        joue ou pas
    """
    score = 0
    pion_adverse = 3 - pion

    if fenetre.count(pion) == 4:
        score += 10000  # Augmenter considérablement le score pour un alignement gagnant
    elif fenetre.count(pion) == 3 and fenetre.count(0) == 1:
        score += 100  # Augmenter le score pour une opportunité presque gagnante
    elif fenetre.count(pion) == 2 and fenetre.count(0) == 2:
        score += 5

    # Stratégie défensive: bloquer l'adversaire
    if fenetre.count(pion_adverse) == 3 and fenetre.count(0) == 1:
        score -= 50  # Augmenter le poids pour bloquer un coup presque gagnant de l'adversaire

    return score

def evaluer_position(plateau, pion):
    """
        Parcourt toutes les lignes, colonnes et diagonales du plateau de jeu pour etablir un score global
        de la position du joueur sur le plateau. Permettra a l'IA d'adapter son style de jeu selon le score obtenu.
    """
    score = 0

    # Stratégie 1: Privilégier le centre
    tableau_centre = [int(i) for i in list(plateau[:, COLONNES // 2])]
    compteur_centre = tableau_centre.count(pion)
    score += compteur_centre * 3

    # Stratégie 2: Évaluer les possibilités horizontales
    for ligne in range(LIGNES):
        tableau_ligne = [int(i) for i in list(plateau[ligne, :])]
        for colonne in range(COLONNES-3):
            fenetre = tableau_ligne[colonne:colonne+4]
            score += evaluer_fenetre(fenetre, pion)

    # Stratégie 3: Évaluer les possibilités verticales
    for colonne in range(COLONNES):
        tableau_colonne = [int(i) for i in list(plateau[:, colonne])]
        for ligne in range(LIGNES-3):
            fenetre = tableau_colonne[ligne:ligne+4]
            score += evaluer_fenetre(fenetre, pion)

    # Stratégie 4: Évaluer les diagonales
    for ligne in range(LIGNES-3):
        for colonne in range(COLONNES-3):
            fenetre = [plateau[ligne+i][colonne+i] for i in range(4)]
            score += evaluer_fenetre(fenetre, pion)

    for ligne in range(LIGNES-3):
        for colonne in range(COLONNES-3):
            fenetre = [plateau[ligne+3-i][colonne+i] for i in range(4)]
            score += evaluer_fenetre(fenetre, pion)

    return score

def est_noeud_terminal(plateau):
    return coup_gagnant(plateau, 1) or coup_gagnant(plateau, 2) or len(colonnes_valides(plateau)) == 0

def minimax(plateau, profondeur, alpha, beta, joueur_maximisant):
    colonnes_possibles = colonnes_valides(plateau)
    est_terminal = est_noeud_terminal(plateau)
    if profondeur == 0 or est_terminal:
        if est_terminal:
            if coup_gagnant(plateau, joueur_ia):
                return (None, 100000000000000)
            elif coup_gagnant(plateau, joueur_humain):
                return (None, -10000000000000)
            else:  # Jeu terminé, pas d'autres mouvements valides
                return (None, 0)
        else:  # Profondeur égale à zéro
            return (None, evaluer_position(plateau, joueur_ia))
    if joueur_maximisant:
        valeur = -math.inf
        colonne = random.choice(colonnes_possibles)
        for col in colonnes_possibles:
            ligne = prochaine_ligne_libre(plateau, col)
            copie_plateau = plateau.copy()
            placer_pion(copie_plateau, ligne, col, joueur_ia)
            nouveau_score = minimax(copie_plateau, profondeur - 1, alpha, beta, False)[1]
            if nouveau_score > valeur:
                valeur = nouveau_score
                colonne = col
            alpha = max(alpha, valeur)
            if alpha >= beta:
                break
        return colonne, valeur
    else:  # Joueur minimisant
        valeur = math.inf
        colonne = random.choice(colonnes_possibles)
        for col in colonnes_possibles:
            ligne = prochaine_ligne_libre(plateau, col)
            copie_plateau = plateau.copy()
            placer_pion(copie_plateau, ligne, col, joueur_humain)
            nouveau_score = minimax(copie_plateau, profondeur - 1, alpha, beta, True)[1]
            if nouveau_score < valeur:
                valeur = nouveau_score
                colonne = col
            beta = min(beta, valeur)
            if alpha >= beta:
                break
        return colonne, valeur

# Boucle principale du jeu
def jouer():
    # Initialisation du jeu pour chaque nouvelle partie
    plateau = np.zeros((LIGNES, COLONNES))
    partie_terminee = False
    tour = 0

    # Demander qui commence en premier
    joueur_commence = int(input("Qui commence en premier ? (1: Joueur, 2: IA) : "))

    while not partie_terminee:
        # Tour du joueur
        if tour % 2 == joueur_commence - 1:
            try:
                colonne = int(input("Sélectionnez votre colonne (0-{}): ".format(COLONNES-1)))
                if colonne not in range(COLONNES) or plateau[LIGNES-1][colonne] != 0:
                    print("Colonne invalide, réessayez.")
                    continue
            except ValueError:
                print("Veuillez entrer un numéro de colonne valide.")
                continue

            ligne = prochaine_ligne_libre(plateau, colonne)
            placer_pion(plateau, ligne, colonne, joueur_humain)

            if coup_gagnant(plateau, joueur_humain):
                afficher_plateau(plateau)
                print("Le joueur gagne !")
                partie_terminee = True

        # Tour de l'IA
        else:
            colonne, score_minimax = minimax(plateau, 5, -math.inf, math.inf, True)
            ligne = prochaine_ligne_libre(plateau, colonne)
            placer_pion(plateau, ligne, colonne, joueur_ia)

            print("L'IA a joué dans la colonne :", colonne)

            if coup_gagnant(plateau, joueur_ia):
                afficher_plateau(plateau)
                print("L'IA gagne !")
                partie_terminee = True

        afficher_plateau(plateau)

        # Changer de tour
        tour += 1


jouer()

Qui commence en premier ? (1: Joueur, 2: IA) : 2
L'IA a joué dans la colonne : 6
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 2 0 0 0 0 0 

Sélectionnez votre colonne (0-11): 5
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 2 0 0 0 0 0 

L'IA a joué dans la colonne : 5
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 2 0 0 0 0 0 0 
0 0 0 0 0 1 2 0 0 0 0 0 

Sélectionnez votre colonne (0-11): 6
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 2 1 0 0 0 0 0 
0 0 0 0 0 1 2 0 0 0 0 0 

L'IA a joué dans la colonne : 3
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 2 1 0 0 0 0 0 
0 0 0 2 0 1 2 0 0 0 0 0 

Sélectionnez votre colonne